# Homework:  Hotel Database -- Due Nov 21.

Design a database with 

- Rooms
- Guests 
- Reservations
- Check-in:  date/time  for reservation
- Check-out: date/time  for reservation

### Rules
1. A reservation reserves one room for one night for one guest
2. A guest can reserve multiple rooms per night.
2. A guest must have a reservation to check in.
3. Checkout can only be done after a check-in. 
4. Checking into a room is allowed only after the previous guest checks out 

### Assignment
1. Design tables that enforce these rules
2. Populate with fake rooms, customers, and nights.
3. Define function `reserve` to make a reservation. Call it to make fake reservations.
3. Define functions `checkin` and `checkout`. Call them to demonstrate that they work. 
4. Use transactions if or when necessary

In [ ]:
schema = dj.schema('dimitri_hotel')

In [ ]:
import random

In [ ]:
Room.drop()

In [ ]:
@schema
class Beds(dj.Lookup):
    definition = """
        beds : varchar(12)
        """
    contents = [
        ['Queen'],
        ['King'],
        ['Two Twins'],
        ['Twin']]


@schema
class Room(dj.Lookup):
    definition = """
    room  : int
    ---
    -> Beds
    """    
    
    @property
    def contents(self):
        beds = Beds.fetch(as_dict=True)
        for room in range(100):
            bed_choice = random.choice(beds)
            yield dict(room=room, **bed_choice)
    

@schema 
class RoomRate(dj.Manual):
    definition = """
    -> Room
    night   : date
    ---
    rate : decimal(5, 2)
    """
    
@schema
class Guest(dj.Manual):
    definition = """
    guest_id  : int
    ---
    phone = null :   varchar(25)
    full_name  : varchar(60)
    unique index(full_name, phone)
    """
    
@schema
class Reservation(dj.Manual):
    definition = """
    -> RoomRate
    ---
    -> Guest
    """

@schema
class CheckIn(dj.Manual):
    definition = """
    -> Reservation 
    ---
    checkin_time = CURRENT_TIMESTAMP : datetime
    """
    
@schema
class CheckOut(dj.Manual):
    definition = """
    -> CheckIn
    ---
    checkout_time = CURRENT_TIMESTAMP : datetime
    """

In [ ]:
import faker
fake = faker.Faker()

In [ ]:
RoomRate.insert((
    dict(key, 
        night=fake.date_this_month(), 
        rate=round(random.uniform(30, 250), 2))
            for key in Room.fetch('KEY')
            for _ in range(10)), skip_duplicates=True)

In [ ]:
RoomRate()

In [ ]:
Guest.insert((dict(guest_id=i, full_name=fake.name(), phone=fake.phone_number()))
             for i in range(100)) 

In [ ]:
Guest()

In [ ]:
def reserve(guest_id, night):
    with dj.conn().transaction:
        available_rooms = ((RoomRate - Reservation) & {'night': night}).fetch(as_dict=True)
        try:
            choice = random.choice(available_rooms)
        except IndexError:
            raise IndexError(f'Sorry, no rooms available for {night}')
        name = (Guest & {'guest_id': guest_id}).fetch1('full_name')
        print('Success. Reserving room {room} at rate {rate} for {name}'.format(
            name=name, **choice))
        Reservation.insert1(dict(choice, guest_id=guest_id), ignore_extra_fields=True)

In [ ]:
reserve(87, '2020-11-23')

In [ ]:
Reservation()

In [ ]:
def checkin(room, night, guest_id):
    with CheckIn.connection.transaction:
        if (CheckIn & {'room': room}) - CheckOut:
            raise RuntimeError('Someone is still in that room')
        if not (Reservation & {'night': night, 'guest_id': guest_id, 'room': room}):
            raise RuntimeError('You are not the person on the reservation')
        CheckIn.insert1(dict(room=room, night=night))

In [ ]:
CheckIn()

In [ ]:
checkin(99, "2020-11-23", 65)

In [ ]:
def checkout(room, night, guest_id):
    with CheckOut.connection.transaction:
        if not (Reservation & {'night': night, 'guest_id': guest_id, 'room': room}):
            raise RuntimeError('You are not the person on the reservation')
        CheckOut.insert1(dict(room=room, night=night))

In [ ]:
checkout(18, "2020-11-23", 87)